# Getting hands dirty

The main thing here to do is, I am going to unify all the reports to one file. So, it is easy for me to do analytics than doing in different files. So, thought of a unified schema that is a superset of all the columns of the reports. This ensures that every piece of information has a place in the final consolidated file.

### Dependencies and data path

Simple ones only.

In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime 

file_path = "/home/ntejha/Projects/PowerTrack/data/raw/CEA/Daily"

### 01_All_India_Region_Wise_Power_generation_Overview

Importing this to dataframe and cleaning it to a usable format and going to append it with the sum_df to concatenate it at the end.

In [2]:
file_01 = "/home/ntejha/Projects/PowerTrack/data/raw/CEA/Daily/01_All_India_Region_Wise_Power_generation_Overview.xlsx"

df = pd.read_excel(file_01, skiprows=2)

df.head(20)

,Unnamed: 0,All India / Regionwise Power Generation Overview 13-Aug-2025,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Sub-Report-1
0,NaN,All India / Regions,NaN,NaN,NaN,NaN,Installed Capacity (MW),Monitored Capacity (MW),Annual Target (MU),Generation 2025-26 (MU),NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Today's Program,Today's Actual,Apr 1 Till Date,NaN,Deviation,NaN,%Deviation
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Program,Actual,NaN,NaN,NaN
3,NaN,Northern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,THERMAL,NaN,NaN,NaN,NaN,"56,196.96","56,196.96","337,626.00",881.29,804.04,"130,106.03","111,400.33","-18,705.70",NaN,-14.38
5,NaN,NUCLEAR,NaN,NaN,NaN,NaN,"2,220.00","2,220.00","15,247.00",39.61,36.62,"5,392.93","4,607.76",-785.17,NaN,-14.56
6,NaN,HYDRO,NaN,NaN,NaN,NaN,"21,174.26","21,174.26","80,988.00",400.14,404.65,"39,608.82","40,430.05",821.23,NaN,2.07
7,NaN,Total,NaN,NaN,NaN,NaN,"79,591.22","79,591.22","433,861.00","1,321.04","1,245.31","175,107.78","156,438.14","-18,669.64",NaN,-10.66
8,NaN,Western,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,THERMAL,NaN,NaN,NaN,NaN,"94,070.40","94,070.40","561,830.00","1,384.04","1,426.14","211,680.68","196,411.41","-15,269.27",NaN,-7.21


In [3]:
experimental_df = df

experimental_df.columns = df.iloc[0]             # use row 0 as base
experimental_df.dropna(axis=1, how='all', inplace=True)

new_cols = experimental_df.columns.tolist()
new_cols[6:] = experimental_df.iloc[1, 6:].tolist()
experimental_df.columns = new_cols

new_cols = experimental_df.columns.tolist()
new_cols[8:10] = experimental_df.iloc[2, 8:10].tolist()
experimental_df.columns = new_cols

experimental_df = experimental_df.drop(experimental_df.columns[[1, 2]], axis=1)
experimental_df = experimental_df.drop(index=[0, 1, 2]).reset_index(drop=True)
experimental_df = experimental_df.drop(experimental_df.index[30:]).reset_index(drop=True)

experimental_df_2 = experimental_df.copy()

regions = ["Northern", "Western", "Southern", "Eastern", "North-Eastern", "All India"]

experimental_df_2["All India / Regions"] = (
    experimental_df_2["All India / Regions"]
    .str.strip()
    .replace({"North Eastern": "North-Eastern", "ALL INDIA": "All India"})
)

experimental_df_2["Region"] = experimental_df_2["All India / Regions"].where(
    experimental_df_2["All India / Regions"].isin(regions)
).ffill()

experimental_df_2["EnergyType"] = experimental_df_2["All India / Regions"].where(
    ~experimental_df_2["All India / Regions"].isin(regions)
)

experimental_df_2.loc[
    experimental_df_2["All India / Regions"].str.upper() == "TOTAL", "EnergyType"
] = "TOTAL"

experimental_df_2.drop(columns=["All India / Regions"], inplace=True)

cols = ["Region", "EnergyType"] + [c for c in experimental_df_2.columns if c not in ["Region", "EnergyType"]]
experimental_df_2 = experimental_df_2[cols].reset_index(drop=True)

experimental_df_2 = experimental_df_2[experimental_df_2["EnergyType"].str.upper() != "TOTAL"].copy()

experimental_df_2 = experimental_df_2[experimental_df_2.isna().sum(axis=1) <= 8]


All_India_Region_Wise_Power_generation_Overview_01 = experimental_df_2.copy()

All_India_Region_Wise_Power_generation_Overview_01.head(33)

,Region,EnergyType,Installed Capacity (MW),Monitored Capacity (MW),Annual Target (MU),Today's Program,Today's Actual,Program,Actual,Deviation,%Deviation
1,Northern,THERMAL,"56,196.96","56,196.96","337,626.00",881.29,804.04,"130,106.03","111,400.33","-18,705.70",-14.38
2,Northern,NUCLEAR,"2,220.00","2,220.00","15,247.00",39.61,36.62,"5,392.93","4,607.76",-785.17,-14.56
3,Northern,HYDRO,"21,174.26","21,174.26","80,988.00",400.14,404.65,"39,608.82","40,430.05",821.23,2.07
6,Western,THERMAL,"94,070.40","94,070.40","561,830.00","1,384.04","1,426.14","211,680.68","196,411.41","-15,269.27",-7.21
7,Western,NUCLEAR,"3,240.00","3,240.00","23,240.00",62.23,48.99,"9,117.34","7,156.65","-1,960.69",-21.51
8,Western,HYDRO,"7,392.00","7,392.00","17,229.00",72.52,104.85,"5,482.76","6,877.59","1,394.83",25.44
11,Southern,THERMAL,"51,241.60","51,231.60","303,516.00",760.98,609.67,"115,765.95","86,992.76","-28,773.19",-24.85
12,Southern,NUCLEAR,"3,320.00","3,320.00","18,105.00",35.00,46.69,"7,404.00","8,900.31","1,496.31",20.21
13,Southern,HYDRO,"13,047.15","13,047.15","34,253.00",136.67,177.63,"10,926.71","15,948.98","5,022.27",45.96
16,Eastern,THERMAL,"39,029.71","39,017.71","285,685.00",742.41,720.67,"106,903.00","94,448.87","-12,454.13",-11.65


In [4]:
sum_df = []

sum_df.append(All_India_Region_Wise_Power_generation_Overview_01)

### 02_Region_Wise_State_Wise_Sector_Wise_TypeWise_Station_Wise_Unit_Wise_Generation_Report

In [5]:
file_02 = "/home/ntejha/Projects/PowerTrack/data/raw/CEA/Daily/02_Region_Wise_State_Wise_Sector_Wise_TypeWise_Station_Wise_Unit_Wise_Generation_Report.xlsx"

df = pd.read_excel(file_02, skiprows=2)
df.head(20)

,POWER STATION,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,MONITORED\nCAP.\n(MW),GENERATION (MU),Unnamed: 9,Unnamed: 10,Unnamed: 11,COAL STOCK\nIN DAYS,CAP. UNDER\nOUTAGE\n(MW)\n\n,OUTAGE DATE\n(DD/MM/YYYY HH24:MI),Unnamed: 15,EXPECTED DATE /\nSYNC. DATE,REMARKS
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TODAY'S\nPROGRAM,TODAY'S\nACTUAL\n,APRIL 1 TILL DATE (2025-26),NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PROG.,ACTUAL,2025-08-13 00:00:00,NaN,NaT,NaN,NaN,NaN
2,NORTHERN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,REGION TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,79591.22,1321.04,1245.31,175107.78,156438.14,NaN,16045.65,NaT,NaN,NaN,NaN
4,DELHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
5,STATE TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,2100.40,16.19,14.27,2400.47,1690.67,NaN,150.00,NaT,NaN,NaN,NaN
6,SECTOR:,NaN,NaN,NaN,STATE SECTOR,NaN,NaN,2100.40,16.19,14.27,2400.47,1690.67,NaN,150.00,NaT,NaN,NaN,NaN
7,TYPE:,NaN,NaN,NaN,THER (GT),NaN,NaN,2100.40,16.19,14.27,2400.47,1690.67,NaN,150.00,NaT,NaN,NaN,NaN
8,I.P.CCPP,NaN,NaN,NaN,NaN,NaN,NaN,270.00,0.96,0.42,146.48,130.95,NaN,150.00,NaT,NaN,NaN,NaN
9,PRAGATI CCGT-III,NaN,NaN,NaN,NaN,NaN,NaN,1500.00,11.36,6.87,1680.68,1011.88,NaN,0.00,NaT,NaN,NaN,NaN


In [6]:
df = df.iloc[:, :14]
df = df.drop(df.columns[12], axis=1)

new_cols = df.columns.tolist()
new_cols[8:10] = df.iloc[0, 8:10].tolist()
df.columns = new_cols

new_cols = df.columns.tolist()
new_cols[10:12] = df.iloc[1, 10:12].tolist()
df.columns = new_cols

df.insert(loc=0, column="Type", value=0)
df.insert(loc=0, column="Sector", value=0)
df.insert(loc=0, column="State", value=0)
df.insert(loc=0, column="Region", value=0)

df = df.drop(index=[0, 1]).reset_index(drop=True)
df = df.drop(df.columns[[5, 7]], axis=1)

df.columns = df.columns.str.strip()
df['POWER STATION'] = df['POWER STATION'].str.strip()

copy_df_02 = df.copy()

regions = ["NORTHERN", "WESTERN", "SOUTHERN", "EASTERN", "NORTH-EASTERN", "CENTRAL"]

states_of_india = [
    "DELHI", "HARYANA", "PUNJAB", "UTTAR PRADESH", "RAJASTHAN", "GUJARAT", 
    "MAHARASHTRA", "MADHYA PRADESH", "CHHATTISGARH", "JHARKHAND", "BIHAR", 
    "WEST BENGAL", "ODISHA", "ASSAM", "MEGHALAYA", "MANIPUR", "MIZORAM", 
    "TRIPURA", "NAGALAND", "ARUNACHAL PRADESH", "SIKKIM", "TAMIL NADU", 
    "KERALA", "KARNATAKA", "TELANGANA", "ANDHRA PRADESH", "GOA"
]

current_region = None
current_state = None
current_sector = None
current_type = None

for idx, row in copy_df_02.iterrows():
    ps = str(row["POWER STATION"]).strip()

    if ps in regions:
        current_region = ps

    elif ps in states_of_india:
        current_state = ps

    elif "SECTOR" in ps.upper():
        current_sector = row.get("Unnamed: 3")

    elif "TYPE" in ps.upper():
        current_type = row.get("Unnamed: 4")

    copy_df_02.at[idx, "Region"] = current_region
    copy_df_02.at[idx, "State"] = current_state
    copy_df_02.at[idx, "Sector"] = current_sector
    copy_df_02.at[idx, "Type"] = current_type

copy_df_02 = copy_df_02[~copy_df_02["POWER STATION"].str.contains("TOTAL|SECTOR|TYPE", na=False)]
copy_df_02 = copy_df_02[~copy_df_02["POWER STATION"].isin(regions)]
copy_df_02 = copy_df_02.reset_index(drop=True)



copy_df_02 = copy_df_02[copy_df_02["POWER STATION"] != "Unit"]

if "Sector" in copy_df_02.columns:
    copy_df_02 = copy_df_02.drop(columns=["Sector"])


state_names = copy_df_02["State"].dropna().unique().tolist()

copy_df_02 = copy_df_02[~copy_df_02["POWER STATION"].isin(state_names)]



copy_df_02 = copy_df_02.drop(copy_df_02.columns[[4,5,6,7]], axis=1)


copy_df_02.columns = (
    copy_df_02.columns
    .str.replace("\n", " ", regex=True)   
    .str.strip()                          
    .str.replace(r"\s+", " ", regex=True) 
    .str.upper()                         
)


Region_Wise_State_Wise_Sector_Wise_TypeWise_Station_Wise_Unit_Wise_Generation_Report_02 = copy_df_02.copy()

Region_Wise_State_Wise_Sector_Wise_TypeWise_Station_Wise_Unit_Wise_Generation_Report_02.head(10)




/tmp/ipykernel_3455/16310275.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NORTHERN' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  copy_df_02.at[idx, "Region"] = current_region
/tmp/ipykernel_3455/16310275.py:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'DELHI' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  copy_df_02.at[idx, "State"] = current_state
/tmp/ipykernel_3455/16310275.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'THER (GT)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  copy_df_02.at[idx, "Type"] = current_type


,REGION,STATE,TYPE,POWER STATION,MONITORED CAP. (MW),TODAY'S PROGRAM,TODAY'S ACTUAL,PROG.,ACTUAL,CAP. UNDER OUTAGE (MW)
1,NORTHERN,DELHI,THER (GT),I.P.CCPP,270.00,0.96,0.42,146.48,130.95,150.00
2,NORTHERN,DELHI,THER (GT),PRAGATI CCGT-III,1500.00,11.36,6.87,1680.68,1011.88,0.00
3,NORTHERN,DELHI,THER (GT),PRAGATI CCPP,330.40,3.87,6.98,573.31,547.84,0.00
4,NORTHERN,DELHI,THER (GT),RITHALA CCPP,0.00,0,0,0,0,0.00
6,NORTHERN,HARYANA,THERMAL,PANIPAT TPS,710.00,12.94,14.21,1786.22,1413.64,0.00
10,NORTHERN,HARYANA,THERMAL,RAJIV GANDHI TPS,1200.00,17.84,19.4,2669.92,1944.77,0.00
13,NORTHERN,HARYANA,THERMAL,YAMUNA NAGAR TPS,600.00,11.54,10.06,1713.02,1389.98,0.00
16,NORTHERN,HARYANA,THERMAL,MAHATMA GANDHI TPS,1320.00,21.04,21.93,3165.52,3065.13,0.00
19,NORTHERN,HARYANA,THERMAL,INDIRA GANDHI STPP,1500.00,24.04,16.41,3592.52,2877.03,500.00
23,NORTHERN,HARYANA,THER (GT),FARIDABAD CCPP,431.59,0.65,0,82.45,127.2,431.59


In [7]:
sum_df.append(Region_Wise_State_Wise_Sector_Wise_TypeWise_Station_Wise_Unit_Wise_Generation_Report_02)

In [8]:
for i, df in enumerate(sum_df):
    print(f"\nDataFrame {i}:")
    print(df.head())  



DataFrame 0:
     Region EnergyType Installed Capacity (MW) Monitored Capacity (MW)  \
1  Northern    THERMAL               56,196.96               56,196.96   
2  Northern    NUCLEAR                2,220.00                2,220.00   
3  Northern      HYDRO               21,174.26               21,174.26   
6   Western    THERMAL               94,070.40               94,070.40   
7   Western    NUCLEAR                3,240.00                3,240.00   

  Annual Target (MU)  Today's Program Today's Actual     Program      Actual  \
1         337,626.00           881.29         804.04  130,106.03  111,400.33   
2          15,247.00            39.61          36.62    5,392.93    4,607.76   
3          80,988.00           400.14         404.65   39,608.82   40,430.05   
6         561,830.00         1,384.04       1,426.14  211,680.68  196,411.41   
7          23,240.00            62.23          48.99    9,117.34    7,156.65   

    Deviation %Deviation  
1  -18,705.70     -14.38  
2     

### 10_Daily_Outage_Report_Coal_Lignite_and_Nuclear

In [28]:
file_03 = "/home/ntejha/Projects/PowerTrack/data/raw/CEA/Daily/10_Daily_Outage_Report_Coal_Lignite_and_Nuclear.xlsx"

df = pd.read_excel(file_03, skiprows=4)

df.head(20)

,State/System,Unnamed: 1,Unnamed: 2,Power Station,Unit No.,Planned Maintenance (MW),Forced Maintenance (Major) (MW),Forced Maintenance (Minor) (MW),Others Maintenance (MW),Date& Time of Maintenance,Expected/sync Date of Return,Reasons/Present Status
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DD/MM/YYYY HH24:MM,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DD/MM/YYYY HH24:MM,NaN,NaN
2,1,NaN,NaN,2,3.0,4.0,5.0,6.0,7.0,8,9,10
3,Northern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Haryana,NaN,NaN,INDIRA GANDHI STPP,1.0,0.0,0.0,500.0,0.0,09/08/2025 14:00,NaN,RESERVE SHUT DOWN / STANDBY UNIT
5,Punjab,NaN,NaN,ROPAR TPS,3.0,0.0,0.0,210.0,0.0,13/08/2025 19:50,NaN,COAL FEEDING PROBLEM IN PLANT
6,Punjab,NaN,NaN,ROPAR TPS,4.0,0.0,0.0,210.0,0.0,10/08/2025 09:22,NaN,FURNACE DRAFT ABNORMAL
7,Punjab,NaN,NaN,ROPAR TPS,6.0,0.0,0.0,210.0,0.0,10/08/2025 07:25,NaN,REHEATER TUBE LEAKAGE
8,Punjab,NaN,NaN,TALWANDI SABO TPP,1.0,0.0,0.0,660.0,0.0,08/08/2025 16:00,NaN,GENERTOR
9,Rajasthan,NaN,NaN,DAE (RAJASTHAN),1.0,0.0,0.0,0.0,0.0,09/10/2004 22:58,NaN,GENERATOR EARTH FAULT


In [ ]:
df = df.drop(index=[0,1,2]).reset_index(drop=True)

df.insert(loc=0, column="Region", value=0)

df.rename(columns={'State/System' : "STATE"}, inplace=True)

regions = ["NORTHERN", "WESTERN", "SOUTHERN", "EASTERN", "NORTH-EASTERN", "CENTRAL"]

current_region = None

for i in range(len(df)):
    val = str(df.loc[i, "STATE"]).strip().upper()

    if val in regions:
        current_region = val
    
    df.loc[i, "Region"] = current_region


df = df.iloc[:152].reset_index(drop=True)

df = df[~df["STATE"].str.contains("Total", case=False, na=False)].reset_index(drop=True)

df = df[~(df["Region"].str.upper() == df["STATE"].str.upper())].reset_index(drop=True)

df = df.drop(df.columns[[2, 3]], axis=1)

Daily_Outage_Report_Coal_Lignite_and_Nuclear_10 = df.copy()

Daily_Outage_Report_Coal_Lignite_and_Nuclear_10.tail(50)

,Region,STATE,Power Station,Unit No.,Planned Maintenance (MW),Forced Maintenance (Major) (MW),Forced Maintenance (Minor) (MW),Others Maintenance (MW),Date& Time of Maintenance,Expected/sync Date of Return,Reasons/Present Status
95,SOUTHERN,Karnataka,KUDGI STPP,2.0,800.0,0.0,0.0,0.0,05/08/2025 00:00,NaN,BOILER OVERHAUL/ ANNUAL MAINTENANCE
96,SOUTHERN,Karnataka,KAIGA,1.0,220.0,0.0,0.0,0.0,31/03/2025 23:11,NaN,MISCELLANEOUS PLANNED MAINTENANCE
97,SOUTHERN,Karnataka,BELLARY TPS,3.0,700.0,0.0,0.0,0.0,21/07/2025 23:35,NaN,BOILER OVERHAUL/ ANNUAL MAINTENANCE
98,SOUTHERN,Karnataka,RAICHUR TPS,1.0,0.0,210.0,0.0,0.0,15/03/2024 00:00,NaN,COAL FEEDING SYSTEM FAILURE.
99,SOUTHERN,Karnataka,RAICHUR TPS,2.0,210.0,0.0,0.0,0.0,10/08/2025 10:49,NaN,BOILER OVERHAUL/ ANNUAL MAINTENANCE
100,SOUTHERN,Karnataka,RAICHUR TPS,5.0,0.0,0.0,210.0,0.0,12/08/2025 16:26,NaN,WATER WALL TUBE LEAKAGE
101,SOUTHERN,Karnataka,ADANI POWER LIMITED UDUPI TPP,1.0,0.0,0.0,600.0,0.0,09/08/2025 00:30,NaN,RSD/LOW SCHEDULE
102,SOUTHERN,Tamil Nadu,NEYVELI NEW TPP,2.0,500.0,0.0,0.0,0.0,16/06/2025 18:12,NaN,OVERHAULING WORKS
103,SOUTHERN,Tamil Nadu,NEYVELI TPS-II,1.0,0.0,210.0,0.0,0.0,21/07/2025 09:02,NaN,MILLING SYSTEM /RC FEEDER PROBLEM
104,SOUTHERN,Tamil Nadu,NEYVELI TPS-II,4.0,210.0,0.0,0.0,0.0,14/07/2025 00:00,NaN,BOILER OVERHAUL/ ANNUAL MAINTENANCE


In [49]:
sum_df.append(Daily_Outage_Report_Coal_Lignite_and_Nuclear_10)

In [50]:
for i, df in enumerate(sum_df):
    print(f"\nDataFrame {i}:")
    print(df.head())  



DataFrame 0:
     Region EnergyType Installed Capacity (MW) Monitored Capacity (MW)  \
1  Northern    THERMAL               56,196.96               56,196.96   
2  Northern    NUCLEAR                2,220.00                2,220.00   
3  Northern      HYDRO               21,174.26               21,174.26   
6   Western    THERMAL               94,070.40               94,070.40   
7   Western    NUCLEAR                3,240.00                3,240.00   

  Annual Target (MU)  Today's Program Today's Actual     Program      Actual  \
1         337,626.00           881.29         804.04  130,106.03  111,400.33   
2          15,247.00            39.61          36.62    5,392.93    4,607.76   
3          80,988.00           400.14         404.65   39,608.82   40,430.05   
6         561,830.00         1,384.04       1,426.14  211,680.68  196,411.41   
7          23,240.00            62.23          48.99    9,117.34    7,156.65   

    Deviation %Deviation  
1  -18,705.70     -14.38  
2     

### 12_Thermal_Nuclear_units_out_of_Grid_for_more_than_15_days

In [71]:
file_04 = "/home/ntejha/Projects/PowerTrack/data/raw/CEA/Daily/12_Thermal_Nuclear_units_out_of_Grid_for_more_than_15_days.xlsx"

df = pd.read_excel(file_04, skiprows=2)

df = df.drop(index=[0, 1, 326]).reset_index(drop=True)

Thermal_Nuclear_units_out_of_Grid_for_more_than_15_days_12 = df.copy()

Thermal_Nuclear_units_out_of_Grid_for_more_than_15_days_12.head(20)

,Station,Unit No,Capacity (MW),Date of Maintanance,Expected date of return,remarks
0,FARIDABAD CCPP,3.0,156.07,2019-01-31 23:55:00,NaN,BOILER OVERHAUL/ ANNUAL MAINTENANCE
1,FARIDABAD CCPP,3.0,156.07,2024-06-28 01:16:00,NaN,CONSERVATION OF MAIN FUEL/RSD
2,FARIDABAD CCPP,3.0,156.07,2024-07-29 12:00:00,NaN,NO/REDUCED IRRIGATION DEMAND
3,FARIDABAD CCPP,3.0,156.07,2024-08-01 12:00:00,NaN,CONSERVATION OF MAIN FUEL/RSD
4,FARIDABAD CCPP,3.0,156.07,2025-07-23 22:38:00,2025-08-13 00:00:00,CONSERVATION OF MAIN FUEL/RSD
5,I.P.CCPP,9.0,30.00,2024-11-10 22:10:00,NaN,TURB. AXIAL SHIFT/ THRUST PAD
6,SURATGARH TPS,3.0,250.00,2023-04-22 06:19:00,NaN,ECONOMISER TUBE LEAKAGE
7,SURATGARH TPS,3.0,250.00,2024-11-28 13:48:00,NaN,FUEL SUPPLY & OTHER MISC. PROBLEMS
8,SURATGARH TPS,6.0,250.00,2024-10-11 01:00:00,NaN,MILLING SYSTEM /RC FEEDER PROBLEM
9,SURATGARH TPS,6.0,250.00,2025-06-02 07:48:00,NaN,RSD/LOW SCHEDULE


In [72]:
sum_df.append(Thermal_Nuclear_units_out_of_Grid_for_more_than_15_days_12)

for i, df in enumerate(sum_df):
    print(f"\nDataFrame {i}:")
    print(df.head())  



DataFrame 0:
     Region EnergyType Installed Capacity (MW) Monitored Capacity (MW)  \
1  Northern    THERMAL               56,196.96               56,196.96   
2  Northern    NUCLEAR                2,220.00                2,220.00   
3  Northern      HYDRO               21,174.26               21,174.26   
6   Western    THERMAL               94,070.40               94,070.40   
7   Western    NUCLEAR                3,240.00                3,240.00   

  Annual Target (MU)  Today's Program Today's Actual     Program      Actual  \
1         337,626.00           881.29         804.04  130,106.03  111,400.33   
2          15,247.00            39.61          36.62    5,392.93    4,607.76   
3          80,988.00           400.14         404.65   39,608.82   40,430.05   
6         561,830.00         1,384.04       1,426.14  211,680.68  196,411.41   
7          23,240.00            62.23          48.99    9,117.34    7,156.65   

    Deviation %Deviation  
1  -18,705.70     -14.38  
2     

### 01_Daily_Coal_Report

In [102]:
file_05 = "/home/ntejha/Projects/PowerTrack/data/raw/CEA/Daily/01_Daily_Coal_Report.xlsx"

df = pd.read_excel(file_05, skiprows=3)

df.head(20)

,Sl. No./State,Unnamed: 1,Mode of Transport,Unnamed: 3,Name of Thermal Power Station/Performance of Utility #,Unnamed: 5,Unnamed: 6,Capacity (MW),Unnamed: 8,Current Month PLF (Tentative),...,Unnamed: 31,Receipt of the day('000 T),Consumption of the day ('000 T),Unnamed: 34,Unnamed: 35,Reasons for critical coal stock/Remarks,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,2,NaN,3,NaN,NaN,4.0,NaN,5,...,NaN,14.0,15.0,NaN,NaN,16,NaN,NaN,NaN,NaN
2,"A. PLANT HAVING COAL LINKAGES, NO LINKAGES AND...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Haryana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,NaN,RAIL,NaN,PANIPAT TPS,NaN,NaN,710.0,NaN,82%,...,NaN,7.8,10.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,NaN,RAIL,NaN,RAJIV GANDHI TPS,NaN,NaN,1200.0,NaN,58%,...,NaN,4.0,14.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,NaN,RAIL,NaN,YAMUNA NAGAR TPS,NaN,NaN,600.0,NaN,59%,...,NaN,6.3,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,HPGCL-Total,NaN,NaN,2510.0,NaN,65%,...,NaN,18.1,32.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Punjab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.insert(loc=0, column="CLASSIFICATION", value=0)
df.rename(columns={'Sl. No./State' : "STATE"}, inplace=True)
df  = df.drop(index=[0,1]).reset_index(drop=True)

plant_types = [
    "A. PLANT HAVING COAL LINKAGES, NO LINKAGES AND COAL BLOCK",
    "B. PLANTS DESIGNED ON IMPORTED COAL",
    "C. NOT IN OPERATION",
    "D. PLANTS BASE ON WASHERY REJECTS"
]

df.loc[df["STATE"].isin(plant_types), "CLASSIFICATION"] = df["STATE"]

df["CLASSIFICATION"] = df["CLASSIFICATION"].replace(0, pd.NA).ffill()



df = df[df["CLASSIFICATION"] != df["STATE"]]
df = df[df["STATE"] != "STATE"]

df.insert(loc=2, column="Unit", value=0)

df.loc[df["STATE"].astype(str).str.fullmatch(r"\d+"), "Unit"] = df["STATE"]

df.loc[df["STATE"].astype(str).str.fullmatch(r"\d+"), "STATE"] = None

df = df[~df["Name of Thermal Power Station/Performance of Utility #"]
          .astype(str)
          .str.contains(r"\bTotal\b", case=False, na=False)]


df["STATE"] = df["STATE"].replace("None", pd.NA)  
df["STATE"] = df["STATE"].ffill()  

df = df[df["Unit"] != 0]

df = df.loc[:, ~df.columns.str.strip().str.startswith("Unnamed")]
df = df.loc[:, ~df.columns.str.strip().str.endswith("Unnamed")]

df.head(20)

,CLASSIFICATION,STATE,Unit,Mode of Transport,Name of Thermal Power Station/Performance of Utility #,Capacity (MW),Current Month PLF (Tentative),Normative Stock Reqd.(Days),Daily Requirement @85% PLF (In '000 Tonnes),Normative Stock Reqd.(In '000 Tonnes'),Actual Stock (In ‘000 Tonnes),% of Actual Stock vis-à-vis Normative Stock,Critical (*),Receipt of the day('000 T),Consumption of the day ('000 T),Reasons for critical coal stock/Remarks
221,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,166,Sea,MUTHIARA TPP,1200.00,21%,21.0,16.083360,337.8,NaN,46%,NaN,0.0,0.0,NaN
222,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,167,Sea,OPG Power Generation Private Limited,414.00,58%,21.0,6.173734,129.6,NaN,80%,NaN,5.1,4.5,NaN
223,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,168,Sea,SALAYA TPP,1200.00,48%,21.0,10.428480,219.0,NaN,215%,NaN,42.9,6.9,NaN
224,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,169,Sea,SHREE CEMENT LTD TPS,300.00,50%,21.0,2.564280,53.8,NaN,0%,*,1.7,1.7,Plant to build up stock through import
225,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,170,Sea,SIMHAPURI TPS,600.00,47%,21.0,6.536160,137.3,NaN,185%,NaN,4.6,4.7,NaN
226,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,171,Sea,THAMMINAPATNAM TPS,650.00,45%,21.0,6.630000,139.2,NaN,0%,*,0.0,0.0,Plant to build up stock through import
227,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,172,Sea,TORANGALLU TPS(SBU-I),260.00,90%,21.0,1.543464,32.4,NaN,0%,*,1.0,1.0,Plant to build up stock through import
228,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,173,Sea,TORANGALLU TPS(SBU-II),600.00,58%,21.0,4.773600,100.2,NaN,0%,*,3.0,3.0,Plant to build up stock through import
229,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,174,Sea,TROMBAY TPS,750.00,67%,21.0,9.363600,196.6,NaN,70%,NaN,6.4,7.6,NaN
230,B. PLANTS DESIGNED ON IMPORTED COAL,IPP,175,Sea,TUTICORIN TPP ST-IV,525.00,55%,21.0,5.601330,117.6,NaN,72%,NaN,0.0,4.8,NaN
